<a href="https://colab.research.google.com/github/meowpunch/LENNA/blob/master/NASR/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

learning algorithm에 쓰이는 objective funciton의 많은 원소들은 모든 feature가 centered around zero, variance in the same order임을 가정하는 경우가 많다.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.get_dummies(pd.read_csv("/content/data0520"), columns=["b_type"])
df

In [ ]:
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, PowerTransformer, QuantileTransformer

In [ ]:
prob = df.columns.difference(['b_type_0', 'b_type_1','latency', 'in_ch'], sort = False)
prob

In [ ]:
in_ch_lin = make_pipeline(StandardScaler(), \
                          QuantileTransformer(n_quantiles = 100, output_distribution = 'normal'))
prob_lin = make_pipeline(MaxAbsScaler(), \
                         QuantileTransformer(n_quantiles = 1000, output_distribution = 'normal'))
latency_lin = make_pipeline(RobustScaler(), \
                            QuantileTransformer(n_quantiles = 1000, output_distribution = 'normal'))

In [ ]:
preprocess = make_column_transformer(
    (in_ch_lin, ['in_ch']),
    (prob_lin, prob),
    (latency_lin, ['latency'])
)

In [ ]:
fitted = pd.DataFrame(preprocess.fit_transform(df), columns = df.columns[0:167])
fitted

In [ ]:
fitted = pd.concat([df.iloc[:,167:], fitted], axis=1)
fitted

In [ ]:
fitted.describe()

In [ ]:
for col in fitted.columns:
  plt.hist(fitted[col], bins = 100)
  plt.xlabel(col)
  plt.show()

In [ ]:
for col in df.columns:
  plt.hist(df[col], bins = 100)
  plt.show()

In [ ]:
for i in range(len(df.columns)):
    plt.scatter(df[df.columns[i]], df['latency'])
    plt.xlabel(df.columns[i])
    plt.ylabel('latency')
    plt.show()

In [ ]:
for i in range(len(df.columns)):
  if i <= len(df.columns) - 2:
    plt.scatter(df[df.columns[i]], df[df.columns[i+1]])
    plt.xlabel(df.columns[i])
    plt.ylabel(df.columns[i+1])
    plt.show()